In [1]:
import json
from concurrent.futures import ThreadPoolExecutor, as_completed
from gridUtils import *
from openweatherUtils import *
from tqdm import tqdm

# 输入路径
nc_path = "results_3di.nc"
# 输出文件路径
output_file = "weather_data.json"

# 生成网格点
points = constructWeatherGrid(500, nc_path)


# 初始化存储数据
timestamps = []
points_data = []

# 定义并行任务
def process_point(point):
    lon, lat = point
    openweather_json = get_hourly_forecast(lat, lon)
    decode_data = decode_hourly_forecast_as_array(openweather_json)
    dt_txt_array = extract_dt_txt_array(openweather_json)
    return {"point": (lon, lat), "data": decode_data, "timestamps": dt_txt_array}

# 实时进度条
progress_bar = tqdm(total=len(points), desc="Processing points", unit="point")

# 使用线程池执行并保存结果
results = []

with ThreadPoolExecutor(max_workers=40) as executor:  # 调整 max_workers 为适当线程数
    future_to_point = {executor.submit(process_point, point): point for point in points}

    for future in as_completed(future_to_point):
        point = future_to_point[future]
        try:
            result = future.result()
            # 将解码后的数据存入 points_data
            points_data.append({"point": result["point"], "data": result["data"]})
            # 初始化 timestamps 仅一次
            if not timestamps:
                timestamps = result["timestamps"]
        except Exception as e:
            print(f"Error processing point {point}: {e}")
        finally:
            progress_bar.update(1)  # 更新进度条

progress_bar.close()  # 关闭进度条

# 写入文件，单独存储 timestamps 和 points_data
output = {"timestamps": timestamps, "points_data": points_data}
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(output, f, indent=4)

print(f"Results have been saved to {output_file}")

Processing points: 100%|██████████| 2880/2880 [00:29<00:00, 98.20point/s] 


Results have been saved to weather_data.json


In [2]:
from openweatherUtils import get_hourly_forecast, decode_hourly_forecast, decode_hourly_forecast_as_array
from datetime import datetime# Sydney coordinates
sydney_lat = -33.8688 
sydney_lon = 151.2093



# Get hourly forecast for Hefei
hefei_forecast = get_hourly_forecast(
    lat=sydney_lat, 
    lon=sydney_lon,
    cnt=10,  # Get 48 hours of forecast
    units="metric"
)

hourly_forecast = decode_hourly_forecast_as_array(hefei_forecast)


if hefei_forecast:
    # Decode the forecast data
    decoded_forecast = decode_hourly_forecast(hefei_forecast)
    
    print("Forecast for Wagga Wagga:")
    print(f"Number of timestamps: {len(decoded_forecast['timestamps'])}")
    print(f"Number of rainfall values: {len(decoded_forecast['rainfall'])}")
    

    # Count non-zero rainfall values in the decoded forecast
    non_zero_rainfall = sum(1 for value in decoded_forecast['rainfall'] if value > 0)
    print(f"\nNumber of non-zero rainfall values in forecast: {non_zero_rainfall}")

    print(hourly_forecast)

    # Print formatted decoded forecast data
    print("\nFormatted Forecast Data:")
    print("Timestamps".ljust(20), "Rainfall (mm)")
    print("-" * 40)


    for ts, rain in zip(decoded_forecast['timestamps'], decoded_forecast['rainfall']):
        print(f"{ts.ljust(20)} {rain:>8.2f}")
    print("\nHourly Rainfall Values:")
    print("Timestamp".ljust(20), "Hourly Rainfall (mm)")
    print("-" * 40)
    for i, ts in enumerate(decoded_forecast['timestamps']):
        # Convert timestamp string to datetime
        dt = datetime.strptime(ts, "%Y%m%d-%H%M%S")
        # Check if it's on the hour (minutes = 00)
        if dt.minute == 0:
            print(f"{ts.ljust(20)} {hourly_forecast[i//2]:>8.2f}")


else:
    print("Failed to fetch forecast data for Hefei")



Forecast for Wagga Wagga:
Number of timestamps: 19
Number of rainfall values: 19

Number of non-zero rainfall values in forecast: 10
[0, 0, 0, 0, 0, 0.24, 0.25, 0.17, 0.21, 0.22]

Formatted Forecast Data:
Timestamps           Rainfall (mm)
----------------------------------------
20250303-150000          0.00
20250303-153000          0.00
20250303-160000          0.00
20250303-163000          0.00
20250303-170000          0.00
20250303-173000          0.00
20250303-180000          0.00
20250303-183000          0.00
20250303-190000          0.00
20250303-193000          0.12
20250303-200000          0.12
20250303-203000          0.12
20250303-210000          0.12
20250303-213000          0.09
20250303-220000          0.09
20250303-223000          0.10
20250303-230000          0.10
20250303-233000          0.11
20250304-000000          0.11

Hourly Rainfall Values:
Timestamp            Hourly Rainfall (mm)
----------------------------------------
20250303-150000          0.00
20250303-16